In [1]:
import os

import numpy as np
import vtk
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy

import pyfocusr
from pyfocusr.vtk_functions import read_vtk_mesh

In [2]:
location_meshes = '../data/'
n_points = '5k'

if n_points == '5k':
    target_filename = 'target_mesh.vtk'
    source_filename = 'source_mesh.vtk'
elif n_points == '15k':
    target_filename = 'target_mesh_15k.vtk'
    source_filename = 'source_mesh_15k.vtk'
    
target_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, target_filename))
source_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, source_filename))

In [3]:
reg = pyfocusr.Focusr(vtk_mesh_target=target_vtk_mesh, 
                      vtk_mesh_source=source_vtk_mesh,
                      n_spectral_features=3,
                      n_extra_spectral=3,
                      get_weighted_spectral_coords=False,
                      list_features_to_calc=[], # 'curvatures', min_curvature' 'max_curvature'
                      rigid_reg_max_iterations=100,
                      non_rigid_alpha=0.01,
                      non_rigid_beta=50,
                      non_rigid_n_eigens=100,
                      non_rigid_max_iterations=300,
                      rigid_before_non_rigid_reg=False,
                      projection_smooth_iterations=1,
                      graph_smoothing_iterations=600,
                      feature_smoothing_iterations=10,
                      include_points_as_features=False,
                      norm_physical_and_spectral=True,
                      feature_weights=np.diag([.1,.1]),
                      n_coords_spectral_ordering=10000,
                      n_coords_spectral_registration=1000,
                      initial_correspondence_type='hungarian',
                      final_correspondence_type='kd')  #'kd' 'hungarian'

Starting Focusr
Starting ICP
Starting to build first graph
Loaded Mesh 1
Beginning Eigen Decomposition
Starting!
All final eigenvalues are: 
[-6.04319128e-18  8.39246263e-04  1.63007145e-03  2.12549101e-03
  3.13941439e-03  3.77495258e-03  4.01682329e-03]
------------------------------------------------------------------------
Final eigenvalues of interest are: 
[0.00083925 0.00163007 0.00212549 0.00313941 0.00377495 0.00401682]
Computed spectrum 1
Loaded Mesh 2
Beginning Eigen Decomposition
Starting!
All final eigenvalues are: 
[-1.07630745e-17  8.31236570e-04  1.64152416e-03  2.11362458e-03
  3.09029787e-03  3.88535401e-03  3.92405051e-03]
------------------------------------------------------------------------
Final eigenvalues of interest are: 
[0.00083124 0.00164152 0.00211362 0.0030903  0.00388535 0.00392405]
Computed spectrum 2


Printed statement indicate progress while loading the meshes and computing spectral coordinates. The eigenvalues are printed, and if there were too many null eigenvalues then the eigenvectors will be re-calculated and the eigenvalues re-printed. 


### View mesh features

### View un-ordered/corrected/weighted eigenvectors (spectral coordinates)

#### Source Eigenvector 1

In [4]:
eig_vec = 2

In [5]:
reg.graph_source.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

#### Target Eigenvector 1

In [6]:
reg.graph_target.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Notes about comparison: 
Depending on if we include features ('curvature') or not, we will get different results for these meshes. 

If we leave the list at `list_features_to_calc` as an empty list `[]` then: <br>
We note that the color gradient is the same (in the same direction) between the two meshes. However, the colours are flipped. This is because the direction of "positive" can be flipped from one mesh to another. This will (should) be corrected automatically before the registration begins. 

If we include `'curvature'` in `list_features_to_calc` with `['curvature']` then: <br>
We get eigenvectors that align in direction and orientation (at least for the first 3 (0, 1, 2). 

Note, the flipped nature of the spectral coordinates can/will be corrected in the algorithm (if it exists). However, this only works because these are both of the same leg (right - as evident by the large side of the trochlear groove) and will likely faily/break if a left and right leg were analyzed. Therefore, all legs should be flipped to be the same side to "fix" things. 

## Begin Registration
This will print out the registration parameters/results as it goes. <br>
- First it will print out the "pairs" of eigenvectors between the two meshes and which ones on the source were flipped to match the target eigenvectors. 
- Second, if rigid registration was performed first, it will print the rigid registration progress and ultimately the registration "results"/"parameters" 
- Third it will print the non-rigid registration progress. 


In [7]:
reg.align_maps()


Eigenvector Sorting Results

The matches for eigenvectors were as follows:
Target	|  Source
     0	|  0     
     1	|  1     
     2	|  -2    
     3	|  3     
     4	|  -4    
     5	|  5     
*Negative source values means those eigenvectors were flipped*
 
Number of features (including spectral) used for registartion: 3

Non-Rigid (Deformable) Registration Beginning

Iteration:1
ML: -8380.101; 	ML change (error):  8380.101; 	Sigma^2:     0.069; 	Sigma^2 change:     0.106
[                                                                        ]
Iteration:2
ML: -9102.485; 	ML change (error):   722.384; 	Sigma^2:     0.061; 	Sigma^2 change:     0.008
[                                                                        ]
Iteration:3
ML: -9149.833; 	ML change (error):    47.349; 	Sigma^2:     0.052; 	Sigma^2 change:     0.009
[                                                                        ]
Iteration:4
ML: -9215.099; 	ML change (error):    65.265; 	Sigma^2:     0.043; 	Sigm

Iteration:43
ML:-11100.008; 	ML change (error):     0.004; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[==========                                                              ]
Iteration:44
ML:-11100.012; 	ML change (error):     0.004; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[==========                                                              ]
Iteration:45
ML:-11100.015; 	ML change (error):     0.003; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[==========                                                              ]
Iteration:46
ML:-11100.018; 	ML change (error):     0.003; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:47
ML:-11100.021; 	ML change (error):     0.003; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:48
ML:-11100.025; 	ML change (error):     0.003; 	Sigma^2:     0.001; 	Sigma^2 c

Iteration:90
ML:-11100.088; 	ML change (error):     0.001; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=====================                                                   ]
Iteration:91
ML:-11100.089; 	ML change (error):     0.001; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=====================                                                   ]
Iteration:92
ML:-11100.089; 	ML change (error):     0.001; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[======================                                                  ]
Iteration:93
ML:-11100.090; 	ML change (error):     0.001; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[======================                                                  ]
Iteration:94
ML:-11100.090; 	ML change (error):     0.001; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[======================                                                  ]
Iteration:95
ML:-11100.091; 	ML change (error):     0.001; 	Sigma^2:     0.001; 	Sigma^2 c

Iteration:137
ML:-11100.101; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[================================                                        ]
Iteration:138
ML:-11100.101; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=================================                                       ]
Iteration:139
ML:-11100.101; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=================================                                       ]
Iteration:140
ML:-11100.101; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=================================                                       ]
Iteration:141
ML:-11100.101; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=================================                                       ]
Iteration:142
ML:-11100.101; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sig

Iteration:183
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========================================                             ]
Iteration:184
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[============================================                            ]
Iteration:185
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[============================================                            ]
Iteration:186
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[============================================                            ]
Iteration:187
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[============================================                            ]
Iteration:188
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sig

Iteration:228
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[======================================================                  ]
Iteration:229
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[======================================================                  ]
Iteration:230
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=======================================================                 ]
Iteration:231
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=======================================================                 ]
Iteration:232
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=======================================================                 ]
Iteration:233
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sig

Iteration:274
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=================================================================       ]
Iteration:275
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[==================================================================      ]
Iteration:276
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[==================================================================      ]
Iteration:277
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[==================================================================      ]
Iteration:278
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[==================================================================      ]
Iteration:279
ML:-11100.103; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sig

## View the spectral coordinates

In [8]:
print('After registration, the range of coordinate values for:\n Source coordinates:\n'
      '{}\n Target coordinates are: \n{}'.format(np.ptp(reg.source_spectral_coords, axis=0),
                                                         np.ptp(reg.target_spectral_coords, axis=0)))


After registration, the range of coordinate values for:
 Source coordinates:
[1. 1. 1.]
 Target coordinates are: 
[1.0029715  0.98284112 1.00942056]


The range of the source are 1.0 because we scaled them to be this. The target are slightly different because they have moved while registering to source coordinates. 

In [9]:
reg.view_aligned_spectral_coords(starting_spectral_coord=0,
                                 include_target_coordinates=True,
                                 include_non_rigid_aligned=True,
                                 include_rigid_aligned=False,
                                 include_unaligned=False,
                                 point_set_colors=None)

Viewer(point_set_colors=array([[0.12156863, 0.46666667, 0.7058824 ],
       [1.        , 0.49803922, 0.0549019…

Above is a 3D view of the spectral (or other) coordinates used for registration. The starting idx for coords can be changed by changing starting_spectral_coord (depending on number of coordinates/features used to register meshes). 

In [10]:
reg.view_meshes_colored_by_spectral_correspondences()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Above is the source mesh (geometry 0) and the target mesh (geometry 1). Target is coloured sequentially (0 to n_points). Source is colored according to the target idx that each source point was registered to. Therefore, colours from the target should be in anatomically the same location as the corresponding colour on the source mesh. <br>

Pink is at the same point in the range of values for both meshes. So, it's alignment between them helps affirm the good registration. 

In [11]:
reg.view_aligned_smoothed_spectral_coords()

Viewer(point_set_colors=array([[0.12156863, 0.46666667, 0.7058824 ],
       [1.        , 0.49803922, 0.0549019…

In the above: 
- Orange is the smoothed target points using their graph. 
- Blue is the source coordinates projected onto the target points.

In [12]:
# reg.get_source_mesh_transformed_weighted_avg()
# reg.
reg.set_all_mesh_scalars_to_corresp_target_idx()

Above will create new mesh where the source mesh points are transformed to their single best fitting/closest point on the target mesh. This will likely result in "doubles". The next set of commands averages these positions to the 3 closest points on target to better diffuse them. 

In [13]:
reg.get_average_shape()

Above will create a new mesh that is the average of the two meshes. 

In [14]:
reg.view_meshes(include_target=True,
                include_source=True,
                include_transformed_target=True,
                include_average=True
                )

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Above is overlapping visualization of the four meshes: 
- Original Target
- Original Source
- Transformed Source (to target surface). 
- Average mesh

Possible to turn each one off/on, turn to wire mesh, change transparency, and change colours. 

### Getting Results: 
- After viewing/analyziny results, we can prob the `reg` object to get various results

In [15]:
target_id_per_source = reg.corresponding_target_idx_for_each_source_pt
transformed_source_nn_xyzs = reg.nearest_neighbor_transformed_points
transformed_source_avg_xyzs = reg.weighted_avg_transformed_points
transformed_source_avg_mesh = reg.get_source_mesh_transformed_weighted_avg
transformed_source_nn_mesh = reg.get_source_mesh_transformed_nearest_neighbour
average_mesh = reg.average_mesh

The above shows some of the key metrics/things that can be extracted. Not all will be always be available (depending on what settings were set). 

`target_id_per_source`:           The target mesh cell/node index that corresponds with each source mesh cell/node<br>

`transformed_source_nn_xyzs`:     The transformed xyz positions for each source node to make it aligned with the target mesh while preserving correct anatomical alignment & *using the nearest neighbor for final alignment*.<br>

`transformed_source_avg_xyzs`:    The transformed xyz positions for each source node to make it aligned with the target mesh while preserving correct anatomical alignment & *using the weighted average alignment method*.<br>

`transformed_source_nn_mesh`:     New mesh with nodes transformed using `transformed_source_nn_xyzs`<br>

`transformed_source_avg_mesh`:    New mesh with nodes transformed using `transformed_source_avg_xyzs`<br>

`average_mesh`:                   New mesh calculated as average of `transformed_source_nn_mesh` & `transformed_source_avg_mesh`
